In [0]:
PFSSPEC_ROOT = '/home/dobos/project/ga_pfsspec_all/'
PFSSPEC_DATA = '/datascope/subaru/data/pfsspec'
GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/grid/phoenix/phoenix_HiRes'

In [0]:
import os, sys, warnings

# Allow load project as module
sys.path.insert(0, os.path.join(PFSSPEC_ROOT, 'python'))

# Set environment
os.environ['PFSSPEC_ROOT'] = PFSSPEC_ROOT
os.environ['PFSSPEC_DATA'] = PFSSPEC_DATA
os.environ['PYSYN_CDBS'] = os.path.join(os.environ['PFSSPEC_DATA'], 'cdbs')

# Filter warnings
warnings.filterwarnings("ignore")

In [0]:
%matplotlib inline

In [0]:
import os
import getpass
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

from scipy.optimize import curve_fit
from scipy import ndimage
from scipy.ndimage.filters import maximum_filter1d
from scipy.interpolate import Rbf, interp1d, interpn

In [0]:
%load_ext autoreload

In [0]:
%autoreload 2

In [0]:
from pfs.ga.pfsspec.core.physics import Physics
from pfs.ga.pfsspec.stellar.continuum import ContinuumModelTrace
from pfs.ga.pfsspec.stellar.continuum.finders import SigmaClipping
from pfs.ga.pfsspec.stellar.continuum.models import PiecewiseChebyshev

# Load grid

In [0]:
from pfs.ga.pfsspec.core.grid import ArrayGrid
from pfs.ga.pfsspec.stellar.grid import ModelGrid
from pfs.ga.pfsspec.stellar.grid.bosz import Bosz
from pfs.ga.pfsspec.stellar.grid.phoenix import Phoenix

In [0]:
fn = os.path.join(os.environ['PFSSPEC_DATA'], GRID_PATH, 'spectra.h5')

In [0]:
#grid = ModelGrid(Bosz(), ArrayGrid)
grid = ModelGrid(Phoenix(), ArrayGrid)
grid.preload_arrays = False
grid.load(fn, format='h5')

In [0]:
for k in grid.grid.axes:
    print(k, grid.grid.axes[k].values)

In [0]:
for k in grid.grid.values:
    print(k, grid.grid.value_shapes[k])

# Pick a spectrum and fit with model

In [0]:
M_H = -1.0
T_eff = 5000
log_g = 3.5
a_M = 0
C_M = 0

In [0]:
idx = grid.array_grid.get_nearest_index(M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M, C_M=C_M)
idx

In [0]:
spec = grid.get_model_at(idx)

In [0]:
spec.M_H, spec.T_eff, spec.log_g, spec.a_M

In [0]:
class Trace(ContinuumModelTrace):
    def on_fit_function_iter(self, piece_id, iter, x, y, w, model, mask):
        if piece_id == 1 and iter < 10:
            plt.figure(figsize=(15, 5))
            
            # plt.plot(x, y, '.', ms=0.3)
            # plt.plot(x[mask], y[mask], '.', ms=1)

            plt.plot(x, np.log(y), '.', ms=0.3)
            plt.plot(x[mask], np.log(y[mask]), '.', ms=1)

            if model is not None:
                #plt.plot(x, model, '-')
                plt.plot(x, np.log(model), '-')
                s = np.std(y[mask] - model[mask])
                print(iter, mask.sum(), s)

            #p = np.polynomial.Chebyshev.fit(x[mask], y[mask] / y.max(), deg=10).coef
            #c = np.polynomial.Chebyshev(p)
            #plt.plot(x, c(x) * y.max(), 'b')

            #plt.xlim(0.3, 0.5)

model = PiecewiseChebyshev(continuum_finder=SigmaClipping(max_iter=50, sigma=[1, 5]), trace=Trace())
model.init_wave(spec.wave)
params = model.fit(spec)

params, params['chebyshev'].shape

In [0]:
model.create_function().get_param_count()

In [0]:
_, cont = model.eval(params)

plt.figure(figsize=(12, 6))

for w in model.photo_limits:
    plt.axvline(w, c='k')

plt.plot(spec.wave, np.log(spec.flux), lw=0.1)
plt.plot(spec.wave, np.log(cont))

plt.xlim(2800, 15000)
plt.ylim(30, None)

In [0]:
_, cont = model.eval(params)

plt.figure(figsize=(12, 6))

for w in model.photo_limits:
    plt.axvline(w, c='k')

plt.plot(spec.wave, np.log(spec.flux) - np.log(cont), lw=0.1)

plt.xlim(2800, 15000)
#plt.ylim(30, None)

# Upper envelope

In [0]:
mask = (4000 < spec.wave) & (spec.wave < 8000)
x = spec.wave[mask]
x = (x - x.min()) / (x.max() - x.min())
y = spec.flux[mask] / spec.flux[mask].max()

size = 501
shift = size // 2
idx = np.arange(shift, x.shape[0] - shift) + (np.arange(size) - shift)[:, np.newaxis]
fx = x[shift:-shift][::shift]
fy = np.max(y[idx], axis=0)[::shift]

plt.figure(figsize=(12, 5))
plt.plot(x, y, '.', ms=0.5)
plt.plot(fx, fy, '.r', ms=2)
print(fx.shape)

p = np.polyfit(fx, fy, 10)
print(p)
plt.plot(x, np.polyval(p, x), '-', ms=0.5)

poly = np.polynomial.Polynomial
p = poly.fit(fx, fy, deg=10).convert().coef
print(p)
plt.plot(x, poly(p)(x), '.', ms=0.5)
# plt.plot(x, np.polyval(p[::-1], x), '.', ms=0.5)